In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('./Data.csv', encoding = "ISO-8859-1")

In [ ]:
df.head()

In [ ]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [ ]:
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)


list1= [i for i in range(25)]
new_Index=[str(i) for i in list1] # Renaming column names for ease of access
data.columns= new_Index
data.head(5)



In [ ]:
for index in new_Index:
    data[index]=data[index].str.lower() #Convertng headlines to lower case because in bag of word model Cat and cat are considered two different words
data.head(2)

In [ ]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))


In [ ]:
headlines[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Naive Bayes / Bag of words
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
#RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

In [ ]:
# Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)


In [ ]:
test

In [ ]:
test.loc[3723,:]

In [ ]:
print(predictions)

In [ ]:
train['headlines'] = headlines
train_headlines = headlines

test_headlines = test_transform

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
train_corpus = []

for i in range(0, len(train_headlines)):


  words = train_headlines[i].split() # Tokenizing the news-title by words


  words = [word for word in words if word not in set(stopwords.words('english'))]


  words = [ps.stem(word) for word in words]


  headline = ' '.join(words)


  train_corpus.append(headline)

In [ ]:
test_corpus = []

for i in range(0, len(test_headlines)):


  words = test_headlines[i].split()


  words = [word for word in words if word not in set(stopwords.words('english'))]


  words = [ps.stem(word) for word in words]

  headline = ' '.join(words)

  test_corpus.append(headline)

In [ ]:
down_words = []
for i in list(train['Label'][train['Label']==0].index):
  down_words.append(train_corpus[i])

up_words = []
for i in list(train['Label'][train['Label']==1].index):
  up_words.append(train_corpus[i])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud1 = WordCloud(background_color='white', width=3000, height=2500).generate(down_words[1])
plt.figure(figsize=(8,8))                                                                                                       #wordcloud for down words
plt.imshow(wordcloud1)
plt.axis('off')
plt.title("Indicating fall in Stock ")
plt.show()

In [ ]:
wordcloud2 = WordCloud(background_color='white', width=3000, height=2500).generate(up_words[5])
plt.figure(figsize=(8,8))
plt.imshow(wordcloud2) #wordcloud for down words
plt.axis('off')
plt.title("Indicating rise in Stock ")
plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test['Label'], predictions)
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
